In [1]:
import os
import glob
import cv2
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.feature import hog
from mpl_toolkits.mplot3d import Axes3D


# Features Extraction

In [112]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    ghist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    bhist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to compute color histogram features  
# Pass the color_space flag as 3-letter all caps string
# like 'HSV' or 'LUV' etc.
def bin_spatial_color_hist(img, color_space='YUV', size=(32, 32)):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    else: feature_image = np.copy(img)             
    
    # color histogram
    hist_features=color_hist(feature_image)
    # Use cv2.resize().ravel() to create the feature vector
    bin_features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return bin_features, hist_features

def extract_hog(img, pix_per_cell=8, cell_per_block=1, orient=9):
    gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    hog_features = hog(gray, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
    return hog_features

def extract_features(img, feature='hog', color_space='YUV', size=(32, 32)):
    hog_features=extract_hog(img)
    if feature=='hog':
        return hog_features
    
    color_bin_features, color_hist_features=bin_spatial_color_hist(img, color_space, size)    
    features=np.concatenate((hog_features,color_bin_features, color_hist_features))
    return features


In [127]:
fpath=os.getcwd()+'/datasets/vehicles/KITTI_extracted/7.png'
img=cv2.imread(fpath)
features=extract_features(img,feature='all')
print(features.shape)


(3744,)


In [182]:
def process_datasets(features='hog', color_space='YUV'):
    if features=='hog':
        feat_size=576
    else:
        feat_size=3744
        
    # Vehicles
    subdir_lists=['KITTI_extracted','GTI_MiddleClose','GTI_Right','GTI_Left','GTI_Far']
    datasets_path=os.getcwd()+'/datasets/vehicles/'
    subdir=[os.path.join(datasets_path,o) for o in os.listdir(datasets_path) 
            if os.path.isdir(os.path.join(datasets_path,o))]

    x = None

    #for subpath in subdir:
    for subpath in subdir_lists:
        print(subpath)
        fcount=len(glob.glob1(datasets_path+subpath,'*.png'))
        print(fcount)
        x_temp=np.empty([fcount,feat_size])
        for i, fname in enumerate(glob.iglob(datasets_path+subpath+'/*.png')):  
            img=cv2.imread(fname)
            x_temp[i]=extract_features(img,features,color_space)
            
        if x is None:
            x=x_temp
        else:
            x=np.vstack((x, x_temp))

    y=np.ones([x.shape[0]], dtype=int)


    # Non-vehicles
    datasets_path=os.getcwd()+'/datasets/non-vehicles/'
    subdir=[os.path.join(datasets_path,o) for o in os.listdir(datasets_path) 
            if os.path.isdir(os.path.join(datasets_path,o))]

    x_non = None

    for subpath in subdir:
        fcount=len(glob.glob1(subpath,'*.png'))
        print(subpath, fcount)
        x_temp=np.empty([fcount,feat_size])
        for i, fname in enumerate(glob.iglob(subpath+'*.png')):    
            img=cv2.imread(fname)
            x_temp[i]=extract_features(img, features, color_space)
        if x_non is None:
            x_non=x_temp
        else:
            x_non=np.vstack((x_non, x_temp))

    y_non=np.zeros([x_non.shape[0]], dtype=int)

    # Stack both vehicle and non-vehicle datasets 
    y=np.hstack((y,y_non))
    x=np.vstack((x,x_non))

    # Normalise features
    X_scaler=StandardScaler().fit(x)
    X=X_scaler.transform(x)

    
    # Shuffle and split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25)

    return X_train, X_test, y_train, y_test



# SVM


In [184]:
X_train, X_test, y_train, y_test=process_datasets(features='hog_and_color', color_space='YUV')

clf=svm.LinearSVC()
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
val_acc=accuracy_score(y_test, y_pred)
print('Validation accuray %.4f'%val_acc)

KITTI_extracted
5966
GTI_MiddleClose
419
GTI_Right
664
GTI_Left
909
GTI_Far
834
C:\Users\soon_\Documents\GitHub\self_driving\vehicle-detect-tracking/datasets/non-vehicles/Extras 5766
C:\Users\soon_\Documents\GitHub\self_driving\vehicle-detect-tracking/datasets/non-vehicles/GTI 3900
Validation accuray 1.0000
